In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import datetime 
import re
from math import floor

In [ ]:
#DetailOutage = pd.read_excel('DetailOutage(13Mar-11Apr).xlsx')
#print(DetailOutage['StartOfOutage'].min())
#print(DetailOutage['StartOfOutage'].max())

In [3]:
DetailOutage2g = pd.read_excel('Input//DetailOutage2G.xlsx')
DetailOutage3g = pd.read_excel('Input//DetailOutage3G.xlsx')
DetailOutage4g = pd.read_excel('Input//DetailOutage4G.xlsx')
NumberOfCellsPerSite = pd.read_excel('Number of Cells Per Site-2G3G4G.xlsx', sheet_name = 'Sheet1')
CellReff = pd.read_excel('Cell Reff.xlsx', sheet_name = 'Sheet1')

In [4]:
# concat 3 technology
DetailOutage = pd.concat([DetailOutage2g, DetailOutage3g, DetailOutage4g], axis = 0, sort = True)
DetailOutage['CellName'] = DetailOutage['CellName'].astype(str)

In [5]:
#Sort by StartOfOutage and then CellName 
DetailOutage = DetailOutage.sort_values(by = ['StartOfOutage'], ascending=True)
DetailOutage = DetailOutage.sort_values(by = ['CellName'], ascending=True)

In [6]:
#Delete incorrect Cell_Names(Numbers, TEST, MCIKANA,...)
pat_name = '\A[A-Z]{2}[1-9]*[GLU]{0,3}[0-9]{4}'#'[A-F]{1}'
DetailOutage['CellName'] = DetailOutage['CellName'].astype(str)
DetailOutage['CellName'] = DetailOutage['CellName'].str.upper()
DetailOutage['Sector'] = DetailOutage['CellName'].apply(lambda x:  ''.join(re.findall(pat_name, x)))
DetailOutage = DetailOutage[DetailOutage['Sector'] != '']
#Create SiteName
DetailOutage['SiteName']=np.where(len(DetailOutage['Sector']) > 7, DetailOutage['Sector'].str[:8], DetailOutage['Sector'].str[:6])
DetailOutage = DetailOutage[['SiteName','CellName','Location','NE','Technology','Province Index','StartOfOutage','EndOfOutage','Down_Time(Seconds)','Hour']]

In [ ]:
DetailOutage['Province Index'] = DetailOutage['SiteName'].str[:2]

In [9]:
pat_tech = '[1-9]{1}[GLU]{1}'
DetailOutage['Location'] = DetailOutage['SiteName'].apply(lambda x: re.sub(pat_tech,'',x))
DetailOutage = DetailOutage[DetailOutage['Location'] != '']

In [10]:
#Create Outage Day
##a = DetailOutage
##a['st'] = pd.to_datetime(DetailOutage['StartOfOutage']).astype(str).str[:11] + '00:00:00'
##DetailOutage['StartOfOutage'] = pd.to_datetime(a['st'])
##Timestamp.combine(date, time)
DetailOutage['Outage Day'] = pd.to_datetime(DetailOutage['StartOfOutage']).dt.date

In [11]:
#Create Down_Count
DetailOutageUp60ns = DetailOutage[DetailOutage['Down_Time(Seconds)'] > 60]
DetailOutageUp60ns['Down_Count'] = 1
DetailOutageUp60ns['Sum_Down_Time'] = DetailOutageUp60ns['Down_Time(Seconds)']
DetailOutage2 = DetailOutageUp60ns.groupby(['CellName','Outage Day'], as_index = False).agg({'Down_Count':'sum','Sum_Down_Time' : 'sum'})
DetailOutageUp60ns = DetailOutageUp60ns.drop(['Down_Count','Sum_Down_Time'], axis = 1)
DetailOutageUp60ns = DetailOutageUp60ns.merge(DetailOutage2, on = ['CellName','Outage Day'], how = 'left')
#Remove Duplicates on CellName and OutageDay
DetailOutageUp60ns = DetailOutageUp60ns.drop_duplicates(subset =['CellName','Outage Day'],keep = 'first')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
############متوسط تعداد قطعی سلهای هر سایت در یک روز 
DetailOutageUp60ns['AVG_Down_Count_Cell_PerDay'] = DetailOutageUp60ns['Down_Count']
DetailOutage3 = DetailOutageUp60ns.groupby(['SiteName','Outage Day'], as_index = False).agg({'AVG_Down_Count_Cell_PerDay':'mean'})
DetailOutageUp60ns = DetailOutageUp60ns.drop(['AVG_Down_Count_Cell_PerDay'], axis = 1)
DetailOutageUp60ns = DetailOutageUp60ns.merge(DetailOutage3, on = ['SiteName','Outage Day'], how = 'left')

In [13]:
#Filter on "Down Count" for more than or equalt to 2
DetailOutageUp60nsUp2DC = DetailOutageUp60ns[DetailOutageUp60ns['Down_Count'] > 1]

In [14]:
#Site Level
##Average Per Cell
###Sort on "Outage Day" from oldest to Newest.
DetailOutageUp60nsUp2DC = DetailOutageUp60nsUp2DC.sort_values(by = ['Outage Day'])
###Sort on "Site Name" from A to Z.
DetailOutageUp60nsUp2DC = DetailOutageUp60nsUp2DC.sort_values(by = ['SiteName'])

In [15]:
#Calculate the "_Fluc_Cellof_each_Site","AVG Down Count PER CELL" and " AVG (Sum Down Time(Second)) PER CELL"
DetailOutageUp60nsUp2DC['Fluc_Cellof_each_Site'] = 1
DetailOutageUp60nsUp2DC_PC = DetailOutageUp60nsUp2DC.groupby(['SiteName','Outage Day'], as_index = False).agg({'Down_Count':'mean','Sum_Down_Time' : 'mean', 'Fluc_Cellof_each_Site' : 'sum'})
DetailOutageUp60nsUp2DC_PerCell = DetailOutageUp60nsUp2DC.drop(['Down_Count','Sum_Down_Time', 'Fluc_Cellof_each_Site'], axis = 1)
DetailOutageUp60nsUp2DC_PerCell = DetailOutageUp60nsUp2DC_PerCell.merge(DetailOutageUp60nsUp2DC_PC, on = ['SiteName','Outage Day'], how = 'left')
#Remove Duplicates on SiteName and OutageDay
DetailOutageUp60nsUp2DC_PerCell = DetailOutageUp60nsUp2DC_PerCell.drop_duplicates(subset =['SiteName','Outage Day'],keep = 'first')

In [16]:
#Add Number Of Cells Per Site
DetailOutageUp60nsUp2DC_PerCell = DetailOutageUp60nsUp2DC_PerCell.merge(NumberOfCellsPerSite, on = ['SiteName'], how = 'left')

In [17]:
# Outage(Site Level Or Cell Level)
DetailOutageUp60nsUp2DC_PerCell['Site or Cell']=np.where(DetailOutageUp60nsUp2DC_PerCell['Fluc_Cellof_each_Site'] > (0.75 * DetailOutageUp60nsUp2DC_PerCell['Cell_Per_Site']), 'SITE', 'CELL')

In [18]:
#Filter on"SITE" 
DetailOutageUp60nsUp2DC_PerCell_SITE = DetailOutageUp60nsUp2DC_PerCell[DetailOutageUp60nsUp2DC_PerCell['Site or Cell'] == 'SITE']

In [19]:
DetailOutageUp60nsUp2DC_PerCell_SITE['NumOfFlucDay'] = 1
DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'] = DetailOutageUp60nsUp2DC_PerCell_SITE['AVG_Down_Count_Cell_PerDay']
DetailOutageUp60nsUp2DC_PerCell_S = DetailOutageUp60nsUp2DC_PerCell_SITE.groupby(['SiteName'], as_index = False).agg({'NumOfFlucDay' : 'sum', 'AvgNumOfOutagePerDay' : 'mean'})
DetailOutageUp60nsUp2DC_PerCell_SITE = DetailOutageUp60nsUp2DC_PerCell_SITE.drop(['NumOfFlucDay', 'AvgNumOfOutagePerDay'], axis = 1)
DetailOutageUp60nsUp2DC_PerCell_SITE = DetailOutageUp60nsUp2DC_PerCell_SITE.merge(DetailOutageUp60nsUp2DC_PerCell_S, on = ['SiteName'], how = 'left')
DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'] = DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'].apply(np.floor)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'] = DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'].apply(np.ceil)

In [21]:
##check 'NumOfFlucDay'  should be less than 30
#DetailOutageUp60nsUp2DC_PerCell_SITE[DetailOutageUp60nsUp2DC_PerCell_SITE['NumOfFlucDay'] > 30]
##check 'AvgNumOfOutagePerDay'  should be less than 24
#DetailOutageUp60nsUp2DC_PerCell_SITE[DetailOutageUp60nsUp2DC_PerCell_SITE['AvgNumOfOutagePerDay'] > 24]

In [22]:
#Remove Duplicates on "Site Name"
DetailOutageUp60nsUp2DC_PerCell_SITE_Loc = DetailOutageUp60nsUp2DC_PerCell_SITE.drop_duplicates(subset =['SiteName'],keep = 'first')

In [23]:
#Filter on "#_Flactuating_Daysof each_Site" for "more than 5"
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Loc['NumOfFlucDay'] > 4]

In [24]:
#Add TAG Number Of Fluctuating Day
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['TAG'] = np.where(DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['NumOfFlucDay'] <= 10,'05 to 10 day',\
                                                     np.where(DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['NumOfFlucDay'] <= 20,'11 to 20 day',\
                                                                                                                                '21 to 30 day'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:
# Add Lat&Long and Category
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.merge(CellReff, on = ['Location'], how = 'left')

In [27]:
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.to_excel('Output//DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.xlsx', index=False, encoding='utf-8-sig')

In [30]:
# Create Proince:
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] = np.where(DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['TAG'] == '05 to 10 day', 1, 0)
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] = np.where(DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['TAG'] == '11 to 20 day', 1, 0)
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] = np.where(DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['TAG'] == '21 to 30 day', 1, 0)

Province = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.groupby(['Province Index'], as_index = False).agg({'Between 05 and 10 days' : 'sum', 'Between 11 and 20 days' : 'sum', 'Between 21 and 30 days' : 'sum'})
Province['Grand Total'] = Province['Between 05 and 10 days'] + Province['Between 11 and 20 days'] + Province['Between 21 and 30 days']
Province.to_excel('Output//DetailOutageUp60nsUp2DC_PerCell_SITE_Province(13Mar_11Apr).xlsx', index=False, encoding='utf-8-sig')

In [31]:
#Create Dashboard
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day[['SiteName','CellName','Location','Province Index','Technology','NumOfFlucDay','AvgNumOfOutagePerDay','TAG','Between 05 and 10 days','Between 11 and 20 days','Between 21 and 30 days','Latitude','Longitude','Category','Site Type','Name']]
###Sort on "AvgNumOfOutagePerDay" from Largest to Smalest.
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.sort_values(by = ['AvgNumOfOutagePerDay'], ascending = False)
###Sort on "NumOfFlucDay" from Largest to Smalest.
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.sort_values(by = ['NumOfFlucDay'], ascending = False)
###Sort on "TAG" from Z to A.
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.sort_values(by = ['TAG'], ascending = False)
#Remove Duplicates on "Location"
DetailOutageUp60nsUp2DC_PerCell_SITE_Locs = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.drop_duplicates(subset =['Location'],keep = 'first')

col = ['Location','2G Site','3G Site','4G Site','Golden Site','City Site','Road Site']
ind = ['Between 05 and 10 days','Between 11 and 20 days','Between 21 and 30 days'] 
Dashboard = pd.DataFrame(columns=col,index=ind)

Dashboard['Location']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Locs.loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Locs['Between 05 and 10 days'] != 0,'Location'].count()
Dashboard['Location']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Locs.loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Locs['Between 11 and 20 days'] != 0,'Location'].count()
Dashboard['Location']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Locs.loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Locs['Between 21 and 30 days'] != 0,'Location'].count()

###2G Site:
Dashboard['2G Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '2G')),'Between 05 and 10 days'].sum()
Dashboard['2G Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '2G')),'Between 11 and 20 days'].sum()
Dashboard['2G Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '2G')),'Between 21 and 30 days'].sum()

###3G Site:
Dashboard['3G Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '3G')),'Between 05 and 10 days'].sum()
Dashboard['3G Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '3G')),'Between 11 and 20 days'].sum()
Dashboard['3G Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '3G')),'Between 21 and 30 days'].sum()

###4G Site:
Dashboard['4G Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '4G')),'Between 05 and 10 days'].sum()
Dashboard['4G Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '4G')),'Between 11 and 20 days'].sum()
Dashboard['4G Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Technology'] == '4G')),'Between 21 and 30 days'].sum()

###Category(Golden and Silver Site):
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Category'] != 'Golden', 'Category'] = 'Silver'
Dashboard['Golden Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Category'] == 'Golden')),'Between 05 and 10 days'].sum()
Dashboard['Golden Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Category'] == 'Golden')),'Between 11 and 20 days'].sum()
Dashboard['Golden Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Category'] == 'Golden')),'Between 21 and 30 days'].sum()

###Site Type(Road and City Site):
DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] != 'Road', 'Site Type'] = 'City'

Dashboard['City Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'City')),'Between 05 and 10 days'].sum()
Dashboard['City Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'City')),'Between 11 and 20 days'].sum()
Dashboard['City Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'City')),'Between 21 and 30 days'].sum()

Dashboard['Road Site']['Between 05 and 10 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 05 and 10 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'Road')),'Between 05 and 10 days'].sum()
Dashboard['Road Site']['Between 11 and 20 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 11 and 20 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'Road')),'Between 11 and 20 days'].sum()
Dashboard['Road Site']['Between 21 and 30 days'] = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day.loc[((DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Between 21 and 30 days'] != 0) & (DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day['Site Type'] == 'Road')),'Between 21 and 30 days'].sum()

Dashboard.to_excel('Output//DetailOutageUp60nsUp2DC_PerCell_SITE_Dashboard(13Mar_11Apr).xlsx', index=True, encoding='utf-8-sig')

In [32]:
#Create Map 
Map = DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day[['Province Index','Location','NumOfFlucDay','AvgNumOfOutagePerDay','TAG','Category','Site Type','Name','Latitude','Longitude']]
Map = Map.sort_values(by = ['TAG'], ascending = False)
Map = Map.drop_duplicates(subset =['Location'],keep = 'first')
Map = Map.sort_values(by = ['Location'], ascending = True)
Map.to_excel('Output//DetailOutageUp60nsUp2DC_PerCell_SITE_Map22.xlsx', index=False, encoding='utf-8-sig')

In [ ]:
#DetailOutageUp60nsUp2DC_PerCell_SITE_Up5Day

In [ ]:
Dashboard

In [ ]:
date = str(pd.to_datetime(DetailOutage['StartOfOutage']).dt.date.max())

In [ ]:
Map.to_excel('Outputs//DetailOutageUp60nsUp2DC_PerCell_SITE_Map(%s).xlsx'%date, index=False, encoding='utf-8-sig')

In [ ]:
col = ['Location','2G Site','3G Site','4G Site','Golden Site','City Site','Road Site']
ind = ['Between 05 and 10 days','Between 11 and 20 days','Between 21 and 30 days'] 
Dashboard = pd.DataFrame(columns=col)

In [ ]:
Dashboard['Location'] = [1,2,3,4]

In [ ]:
Dashboard